In [ ]:
class Config():
    ORIGINAL_IMAGE_SIZE = 512
    IMAGE_SIZE = 288
    TARGET_COLUMNS = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
    N_TARGETS = len(TARGET_COLUMNS)
    BATCH_SIZE = 24
    BATCH_SIZE_VAL = 
    